In [1]:
import observation
from kafka import KafkaProducer

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (__config__.py, line 5)

In [ ]:
NEW_LINE = '\n'
SLEEP = 2
NUMBER_MESSAGES = 50

KAFKA_BROKERS = "sandbox.hortonworks.com:6667"
KAFKA_TOPIC = "test1"

KafkaProducer = KafkaProducer(bootstrap_servers=KAFKA_BROKERS)

while 1:
    for _ in range(NUMBER_MESSAGES):
        obs = observation.Observation()

        msh = 'MSH|^~\&|||||{0}||ORU^R01|||2.3||||'.format(
                    obs.msh_time.strftime("%Y%m%d%H%M%S"))
        
        HL7_MESSAGE = msh

        obr = 'OBR||||{0}||{1}||{2}||||||||||{3}|||||||{4}||||'.format(
            obs.universal_service_id,
            obs.req_time.strftime("%Y%m%d%H%M%S"),
            obs.obs_end_time.strftime("%Y%m%d%H%M%S"),
            obs.method,
            obs.result_status)
        
        for k in obs.oru['components'].keys():
            obx_i = 'OBX|||{0}||{1}|{2}||||||||{3}||||{4}|'.format(
                        k,
                        obs.oru['components'][k]['value'],
                        obs.oru['components'][k]['unit'],
                        obs.obx_time.strftime("%Y%m%d%H%M%S"),
                        obs.oru['method']
                        )
            obr = NEW_LINE.join([obr, obx_i])
            
        HL7_MESSAGE = NEW_LINE.join([HL7_MESSAGE, obr])

        # send HL7_MESSAGE to Kafka
        print (HL7_MESSAGE)
        KafkaProducer.send(KAFKA_TOPIC, str.encode(HL7_MESSAGE))
        KafkaProducer.flush()

    time.sleep(SLEEP)
